# AoC 2022 Day 16 - Volcanos

## Part one

Given a network of valves and tunnels, and a time constraint, what is the max pressure that can be released?

- moving through one tunnel takes 1 minute
- opening takes 1 minute
- 30 minutes total

In [1]:
def load_input(fp):
    with open(fp) as f_in:
        for line in f_in.read().splitlines():
            yield line

In [2]:
# max for test = 1651
test = [line for line in load_input("test.txt")]

In [3]:
test[:3]

['Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
 'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
 'Valve CC has flow rate=2; tunnels lead to valves DD, BB']

Build a graph from the inputs.

- each node has at least one children
- each node has a `release` value
- graph may be cyclic; one's children node may eventually lead back to the original node

In [43]:
class Valve:
    def __init__(self, name='', rate=0, neighbors=None):
        """To use in defaultdict, must set defaults"""
        self.rate = rate
        if neighbors:
            self.neighbors = set(neighbors)
        else:
            self.neighbors = None
            
        
    def __repr__(self):
        return f"rate: {self.rate}\tneighbors: {self.neighbors}"
            

In [31]:
foo = Valve('AA', 3, ['DD', 'II', 'BB'])
print(foo.name, foo.rate, foo.neighbors)

AA 3 {'II', 'BB', 'DD'}


In [11]:
line = test[0]
spl = line.split(" ")
spl

['Valve',
 'AA',
 'has',
 'flow',
 'rate=0;',
 'tunnels',
 'lead',
 'to',
 'valves',
 'DD,',
 'II,',
 'BB']

In [12]:
import re

In [19]:
c = re.compile('Valve ([A-Z]{2}).*rate=(\d+).*valves (.*)')
child_c = re.compile('[A-Z]{2}')
res = c.findall(line)
child_c.findall(res[0][-1])

['DD', 'II', 'BB']

In [21]:
v = re.compile('[A-Z]{2}')
r = re.compile('rate=(\d+)')
v_m = v.findall(line)
r_m = r.findall(line)
print(v_m, r_m)

['AA', 'DD', 'II', 'BB'] ['0']


In [22]:
def parse_valve_tunnel(line: str) -> list:
    """Parse input to return 
    - valve name: str
    - rate: int
    - children: list[str]
    """
    v = re.compile('[A-Z]{2}')
    r = re.compile('rate=(\d+)')
    v_m = v.findall(line)
    r_m = r.findall(line)
    return v_m[0], int(r_m[0]), v_m[1:]
    

In [23]:
parsed = [parse_valve_tunnel(line) for line in test]
parsed[:3]

[('AA', 0, ['DD', 'II', 'BB']),
 ('BB', 13, ['CC', 'AA']),
 ('CC', 2, ['DD', 'BB'])]

In [32]:
nodes = [Valve(*parsed_line) for parsed_line in parsed]
print(nodes[:3])

[Valve: AA	rate: 0	leads to: {'II', 'BB', 'DD'}, Valve: BB	rate: 13	leads to: {'AA', 'CC'}, Valve: CC	rate: 2	leads to: {'BB', 'DD'}]


In [34]:
foo = set()
foo.update([2])
foo

{2}

`defaultdict(list)` is better.

- `graph['AA'] = [neighbors]`
- `rate['AA'] = rate_AA`

To do so with a class...

```py
graph['AA'].neighbors = [neighbors]
graph['AA'].rate = rate_AA
```

perhaps a `defaultdict(node)`.

In [35]:
from collections import defaultdict

tunnels = defaultdict(Valve)

In [36]:
for line in parsed:
    tunnels[line[0]] = Valve(*line)

In [44]:
faa = defaultdict(Valve)
for (name, rate, neighs) in parsed:
    faa[name].rate = rate
    faa[name].neighbors = neighs

In [47]:
faa.get('AA').neighbors

['DD', 'II', 'BB']

In [37]:
tunnels.keys()

dict_keys(['AA', 'BB', 'CC', 'DD', 'EE', 'FF', 'GG', 'HH', 'II', 'JJ'])

In [48]:
tunnels['AA'].neighbors

{'BB', 'DD', 'II'}

In [49]:
from functools import reduce


In [56]:
valve_time = {
    'BB': 1,
    'DD': 5
}
 
print(tunnels['BB'].rate, tunnels['DD'].rate)

13 20


In [57]:
tot = reduce(
    lambda subtot, valve: subtot + valve_time[valve] * tunnels[valve].rate, 
    valve_time, 0)
tot

113

##